# Exploring Hacker News Posts

In this project we'll work with a dataset of submissions to popular technology site [Hacker News](https://news.ycombinator.com/)

Hacker News is a site where user-submitted stories (known as "posts") receive votes and comments, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of the Hacker News listings can get hundreds of thousands of visitors as a result.

For this project, we're specifically interested in posts with titles that begin with either `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the Hacker News community a specific question. Likewise, users submit `Show HN` posts to show the Hacker News community a project, product, or just something interesting. 

Our aim is to determine: 
- Do `Ask HN` or `Show HN` receive more comments on average?
- Do posts created at a certain time receive more comments on average? 

# Introduction 

We'll start by reading the data and removing the headers.

In [2]:
from csv import reader

opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
headers = hn[0]
hn = hn[1:]

print(headers)
print('\n')
for row in hn[:5]:
    print(row)
    print('\n')

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']




# Filtering Our Data

Now, as we're only concerned with the post titles with `Ask HN` OR `Show HN`, we'll create new lists of lists containing just the data for those titles. 

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


After filtering our data, we can see that there are exactly 1744 `Ask HN` posts, 1162 `Show HN` posts, and 17194 other posts that do not belong to any of the previous groups.

# Calculating the Average Number of Comments for Ask HN and Show HN posts

Now that we grouped the ask posts and show posts on different lists, we'll calculate the average number of comments each type of posts receives.

In [4]:
#Calculating the average number of comments for Ask Posts
total_ask_comments = 0

for post in ask_posts:
    num_comments = int(post[4])
    total_ask_comments += num_comments
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)


#Calculating the average number of comments for Show Posts
total_show_comments = 0

for post in show_posts:
    num_comments = int(post[4])
    total_show_comments += num_comments
    
avg_show_comments = total_show_comments /len(show_posts)
print(avg_show_comments)




    

14.038417431192661
10.31669535283993


Here we see that `Ask HN` posts receive more comments, around 14, on average contrary to `Show HN` posts that actually receive approximately 10 comments. Having this in mind, now we'll focus the rest of our analysis on just these kind of posts (`Ask HN` posts)

# Finding the Amount of Ask Posts and Comments by Hour Created

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:
- Calculate the number of ask posts created in each hour of the day, along with the number of comments received.
- Calculate the average number of comments ask posts receive by hour created.

We start with the implementation of the first step:

In [5]:
import datetime as dt

result_list = []

for post in ask_posts:
    result_list.append(
        [post[6], int(post[4])]
    )

comments_by_hour = {}
counts_by_hour = {}

for each_row in result_list:
    date = each_row[0]
    comment = each_row[1]
    time = dt.datetime.strptime(date, "%m/%d/%Y %H:%M")
    time = time.strftime("%H")
    if time in counts_by_hour:
        comments_by_hour[time] += comment
        counts_by_hour[time] += 1
    else:
        comments_by_hour[time] = comment
        counts_by_hour[time] = 1

comments_by_hour


{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

Next, we'll use these two dictionaries `counts_per_hour` & `comments_per_hour` to calculate the average number of comments for posts created during each hour of the day.



In [6]:
avg_by_hour = []

for row in comments_by_hour:
    avg_by_hour.append([row, comments_by_hour[row]/counts_by_hour[row]])
                        
print(avg_by_hour)

[['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['02', 23.810344827586206], ['18', 13.20183486238532], ['03', 7.796296296296297], ['05', 10.08695652173913], ['19', 10.8], ['01', 11.383333333333333], ['22', 6.746478873239437], ['08', 10.25], ['04', 7.170212765957447], ['00', 8.127272727272727], ['06', 9.022727272727273], ['07', 7.852941176470588], ['11', 11.051724137931034]]


Though we have the data we need, this format makes it difficult to identify the hours with the highest values. So, we'll finish this by sorting the list of lists and printing the five highest values in a format that easier to read


In [7]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
# print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print(sorted_swap)

[[38.5948275862069, '15'], [23.810344827586206, '02'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [13.20183486238532, '18'], [11.46, '17'], [11.383333333333333, '01'], [11.051724137931034, '11'], [10.8, '19'], [10.25, '08'], [10.08695652173913, '05'], [9.41095890410959, '12'], [9.022727272727273, '06'], [8.127272727272727, '00'], [7.985294117647059, '23'], [7.852941176470588, '07'], [7.796296296296297, '03'], [7.170212765957447, '04'], [6.746478873239437, '22'], [5.5777777777777775, '09']]


In [62]:
# Sort the values and print the the 5 hours with the highest average comments.

print("Top 5 Hours for 'Ask HN' Comments")
for avg_comments, hour in sorted_swap[:5]:
    print(
        '{}: {:.2f} average comments per post'.format(
            dt.datetime.strptime(hour, "%H").strftime('%H:%M'), avg_comments
        )
    )

Top 5 Hours for 'Ask HN' Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

# Calculating the average number of points for Ask HN posts and Show HN posts

As an extra analysis, we decided to count the number of points, whether `Ask HN` or `Show HN`posts receive more points on average, and do posts created at a certain time receive more points on average.

The number of points in the dataset represent the calculated result of the total number of upvotes a post receive minus the total number of downvotes.

In [27]:
#Calculating the average number of points for Ask Posts
total_ask_points = 0

for post in ask_posts:
    num_of_points = int(post[3])
    total_ask_points += num_of_points
    
avg_ask_points = total_ask_points / len(ask_posts)
print(avg_ask_points)



#Calculating the average number of points for Show Posts
total_show_points = 0

for post in show_posts:
    num_of_points = int(post[3])
    total_show_points += num_of_points
    
avg_show_points = total_show_points / len(show_posts)
print(avg_show_points)

15.061926605504587
27.555077452667813


Here we see that `Show HN` posts receive more points, around 27.56, on average contrary to `Ask HN` posts that actually receive approximately 15 points. Now, just like the previous example with the comments, we'll focus the rest of our analysis on just these kind of posts (`Show HN` posts)

# Finding the Amount of Show Posts and Points by Hour Created

Next, we'll determine if show posts created at a certain time are more likely to receive more points. We'll use the following steps to perform our analysis:
 - Calculate the number of show posts created in each hour of the day, along with the number of points received.
 - Calculate the average number of points that show posts receive by hour created.
 
We start with the implementation of the first step:

In [44]:
result_list = []

for post in show_posts:
    result_list.append([post[6], int(post[3])])
    
    
points_by_hour = {}
counts_by_hour = {}

for row in result_list:
    date = row[0]
    points = row[1]
    
    time = dt.datetime.strptime(date, '%m/%d/%Y %H:%M')
    time = time.strftime('%H')
    
    if time not in counts_by_hour:
        counts_by_hour[time] = 1
        points_by_hour[time] = points
    
    else:
        counts_by_hour[time] += 1
        points_by_hour[time] += points
        
points_by_hour
    

{'14': 2187,
 '22': 1856,
 '18': 2215,
 '07': 494,
 '20': 1819,
 '05': 104,
 '16': 2634,
 '19': 1702,
 '15': 2228,
 '03': 679,
 '17': 2521,
 '06': 375,
 '02': 340,
 '13': 2438,
 '08': 519,
 '21': 866,
 '04': 386,
 '11': 1480,
 '12': 2543,
 '23': 1526,
 '09': 553,
 '01': 700,
 '10': 681,
 '00': 1173}

Next, we'll use these two dictionaries `counts_per_hour` and `points_per_hour` to calculate the average number of points for posts created during each hour of the day.

In [51]:
avg_points_by_hour = []

for hr in points_by_hour:
    avg_points_by_hour.append([hr, points_by_hour[hr] / counts_by_hour[hr]])
    
print(avg_points_by_hour)

[['14', 25.430232558139537], ['22', 40.34782608695652], ['18', 36.31147540983606], ['07', 19.0], ['20', 30.316666666666666], ['05', 5.473684210526316], ['16', 28.322580645161292], ['19', 30.945454545454545], ['15', 28.564102564102566], ['03', 25.14814814814815], ['17', 27.107526881720432], ['06', 23.4375], ['02', 11.333333333333334], ['13', 24.626262626262626], ['08', 15.264705882352942], ['21', 18.425531914893618], ['04', 14.846153846153847], ['11', 33.63636363636363], ['12', 41.68852459016394], ['23', 42.388888888888886], ['09', 18.433333333333334], ['01', 25.0], ['10', 18.916666666666668], ['00', 37.83870967741935]]


As a final step we'll finish by sorting the list of lists and printing the five highest values in a format that easier to read

In [52]:
sorted_avg_points_by_hour = sorted(avg_points_by_hour, reverse=True)
print(sorted_avg_points_by_hour)

[['23', 42.388888888888886], ['22', 40.34782608695652], ['21', 18.425531914893618], ['20', 30.316666666666666], ['19', 30.945454545454545], ['18', 36.31147540983606], ['17', 27.107526881720432], ['16', 28.322580645161292], ['15', 28.564102564102566], ['14', 25.430232558139537], ['13', 24.626262626262626], ['12', 41.68852459016394], ['11', 33.63636363636363], ['10', 18.916666666666668], ['09', 18.433333333333334], ['08', 15.264705882352942], ['07', 19.0], ['06', 23.4375], ['05', 5.473684210526316], ['04', 14.846153846153847], ['03', 25.14814814814815], ['02', 11.333333333333334], ['01', 25.0], ['00', 37.83870967741935]]


In [54]:
#Print the 5 hours with the highest average points.

print("Top 5 Hours for 'Show HN' Points")
for hour, avg_points in sorted_avg_points_by_hour[:5]:
    print(
        '{}: {:.2f} average points per post'.format(
            dt.datetime.strptime(hour, "%H").strftime('%H:%M'), avg_points
        )
    )

Top 5 Hours for 'Show HN' Points
23:00: 42.39 average points per post
22:00: 40.35 average points per post
21:00: 18.43 average points per post
20:00: 30.32 average points per post
19:00: 30.95 average points per post


The hour that receives the most points per post on average is 23:00, with an average of 42.39 points per post.

# Conclusion

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments and points on average. 

Based on our analysis, to maximize the amount of comments that a post receives, we'd recommend the post be categorized as `Ask HN` post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

Whereas, as for the amount of points that a post receives, the post is recommended to be under the `Show HN` post category and created between 19:00 and 23:00 (7:00 pm est - 11:00 pm est).
